# Project 2 report

In [ ]:
# Imports
import numpy as np
import pandas as pd
import folium as fol
import seaborn as sns
import matplotlib.pyplot as plt
import plotly

# Params
pd.set_option('display.max_columns', None)

# Q0: Basic EDA + Tidying

In [ ]:
# Read in data
df = pd.read_csv('dft_rawcount_region_id_3.csv')

In [ ]:
df.head()

In [ ]:
df.describe(include='all')

### Removing unnecessary columns

In [ ]:
df_tidy = df.copy() # Tidy dataset storage

# Cols to drop
rm_cols = ['region_id', 'region_name', 'local_authority_id', 'road_type',
           'start_junction_road_name', 'end_junction_road_name', 'direction_of_travel',
           'year', 'easting', 'northing', 'link_length_miles', 'hgvs_2_rigid_axle',
           'hgvs_3_rigid_axle', 'hgvs_3_or_4_articulated_axle',
           'hgvs_4_or_more_rigid_axle', 'hgvs_5_articulated_axle',
           'hgvs_6_articulated_axle']

# Drop cols
df_tidy.drop(rm_cols, axis=1, inplace=True)

### Renaming Vehicle Labels 

In [ ]:
new_names = ['bikes', 'motorcycles', 'cars', 'buses', 'lgvs', 'hgvs'] # More interpretable labels
name_mappings = dict(zip(df_tidy.columns[8:-1], new_names)) # Creating dict of label mappings
df_tidy.rename(columns=name_mappings, inplace=True) # Renaming columns

### Tidy Transformations

In [ ]:
# all_motor_vehicles --> all_vehicles (include bikes)
df_tidy['all_vehicles'] = df_tidy['bikes'] + df_tidy['motorcycles'] + df_tidy['cars'] + df_tidy['buses'] + \
                          df_tidy['lgvs'] + df_tidy['hgvs']

# New column for specifying whether road is: M, A, B, C, U
df_tidy = df_tidy.assign(
            road_type = lambda dataframe: dataframe['road_name'].map(lambda road_name: road_name[0]))

# Convert count_date to datetime format
df_tidy['count_date'] = pd.to_datetime(df['count_date'], format='%Y-%m-%d')

### Adding Population Data

In [ ]:
# Read in population statistics
pop_df = pd.read_csv('scotland-mid-year-pop-est-19.csv', header=None)

# Rename columns
pop_df.rename(columns={0: 'local_authority_name', 1: 'population'}, inplace=True)

# Convert population to int
pop_df.population = pop_df.population.str.replace(',', '')
pop_df.population = pop_df.population.astype('int') 

# String replaces to avoid join issues
pop_df.local_authority_name = pop_df.local_authority_name.str.replace(' and ', ' & ')
pop_df.local_authority_name = pop_df.local_authority_name.str.replace('Na h-Eileanan Siar', 'Comhairle nan Eilean Siar')

# Overwrite df_tidy with full outer join with population data
df_tidy = pd.merge(df_tidy, pop_df, on='local_authority_name', how='outer')

In [ ]:
df_tidy

# Q1: Bar chart of [X] longest roads showing start and end locations

### Method
* Filter by distinguishing road name 
* Ignore U and C roads 
* We have start and end junctions (general locations)
* How do we calculate distance?
    1. Add up link lengths
    2. Verify with Google Maps



# Q2: Local authorities managing number of roads - mapviz - fixed point in time

### Outcome
* 2019 map of roads managed
* Static map of Scotland with gradient colour scheme

### Method
* Group by `local_authority_name`, `count_id`
* Break down total number of roads per authority into `road_names`
* Possible weight matrix to give larger, busier roads a larger influence in the map - don't want to give U and C thes same weighting as M and A roads
* In `road_name` variable, cut string such that it's only a single character

### Stretch and Challenge
* Maintenance data to observe which authority spends the most on maintainance

### Extra Packages:
* folium
* geopandas

# Q3: Interactive visualization of the distribution of vehicles over each type of road over the past [X] years

### Outcome
* x-axis: discrete vehicle type data e.g.  cars, bikes, buses, hgvs, lgvs etc
* y-axis: type of road e.g. M, A, B, C, U
* Scatter point for each category where size indicates prevalence of that type of vehicle on that type of road
* Slider which shows variation of data over [X] years


### Method
* Group C and U roads
* Normalize scatter points by \pi*R^2


### Stretch and Challenge
* We have dotted line outline of average of previous 5 years giving the reader some indication of growth/reduction
* x-axis: as emojis


### Extra packages
* plotly - used to vary the datapoints



In [ ]:
df_tidy

# Q4: Time of day viz?

### Outcome
* x-axis: Hours of the day (07:00 - 18:00)
* y-axis-1: `all_vehicles`
* y-axis-2: types of vehicle (faceted plot)
* Light alpha filled densities

### Method
* Get hours day
* Group vehicle count by hour type
* Further subset those counts by vehicle

In [ ]:
tod_df = df_tidy[['hour'] + new_names] # Extracting relevant variables
tod_df = tod_df.groupby('hour').sum() # Find total counts of each vehicle at every hour
tod_df.reset_index(inplace=True)
tod_df = tod_df.melt(id_vars='hour', var_name='vehicle_type') # Pivot longer
tod_df.vehicle_type = tod_df.vehicle_type.astype('category') 

In [ ]:
plt.style.use('fivethirtyeight')
g = sns.FacetGrid(data=tod_df, col='vehicle_type', height=3, col_wrap=1, 
                  sharex=True, sharey=False, aspect=5)
g = g.map_dataframe(sns.lineplot, x='hour', y='value', hue ='vehicle_type')

In [ ]:
x = tod_df.hour.unique()
y1 = tod_df.loc[tod_df.vehicle_type=='cars']['value']
y2 = tod_df.loc[tod_df.vehicle_type=='lgvs']['value']
y3 = tod_df.loc[tod_df.vehicle_type=='hgvs']['value']
y4 = tod_df.loc[tod_df.vehicle_type=='buses']['value']
y5 = tod_df.loc[tod_df.vehicle_type=='motorcycles']['value']
y6 = tod_df.loc[tod_df.vehicle_type=='bikes']['value']
y = [y1, y2, y3, y4, y5, y6]

In [ ]:
fig, axes = plt.subplots(6, 1, sharex=True, sharey=False, figsize=(18, 18))
hues = ['#e5ae38', '#8b8b8b', '#6d904f', '#fc4f30', '#810f7c', '#008fd5']
for i, ax in enumerate(axes):
    sns.lineplot(data=tod_df, x='hour', y=y[i], ax=ax, hue='vehicle_type')
    ax.fill_between(x, y[i], color=hues[i], alpha=0.35)
    ax.get_legend().remove()

### TODOs for Q4:

* Add all hours to x-axis
* Format y-axis, and change labels
* Add hover on hour points to read values
* Display legend of colour <--> vehicle_type

# Q5: Bikes per capita per local authority 

### Caveats
1. Count data of bikes is assumed to be representative of the prevalence of bikes in the authority
2. A bike count within a local authority maps to that bike actually belonging to that authority
3. Double counting effects are ignored

This is because bikes are not registered vehicles and hence this assumption has to be made. We think this is a reasonable assumption since bikes are used for micro-mobility in the majority of cases.

### Outcome
* Horizontal bar chart ordered in descending order

### Method
* Get population data and store in tidy dataframe
* Plot the data

## Additional
* Encode or visualise population data



In [ ]:
bikes_cols = ['local_authority_name', 'bikes', 'population']

df_bikes = df_tidy[bikes_cols]

In [ ]:
def div_series(df):
    return df['bikes'].sum() / float(df['population'].sum())

In [ ]:
pop_df

In [ ]:
bikes_per_capita = df_bikes.groupby(['local_authority_name'], as_index=True).apply(div_series).sort_values(ascending=False)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize = (10, 25))
#plt.style.use('fivethirtyeight')
sns.color_palette('mako', as_cmap=True)
sns.barplot(x = bikes_per_capita, y = bikes_per_capita.index)
plt.show()

In [ ]:
bikes_per_capita.index

In [ ]:
# BONUS!

# Create tidy dataframe for bikes only

df_tidy.head()

# Extract: bikes, count_date, hour
bikes_cols = ['count_date', 'hour', 'bikes']
df_bikes = df_tidy[bikes_cols]

# Transform: count_date --> weekday
df_bikes['weekday'] = df_bikes['count_date'].dt.day_name()
df_bikes.groupby(['weekday', 'hour']).sum()

In [ ]:
sns.set_style()